In [18]:
from __future__ import print_function
import os
import sys

In [19]:
#sc = SparkContext()
#sqlContext = SQLContext(sc)
sc

In [20]:
df = sqlContext.read.load("/tmp/idigbio-media-scrap-20170611T145102.parquet")

In [21]:
print(df.count())
df.head(2)

177596


[Row(accessuri='https://botanydb.colorado.edu/COLO_V/00806/00806653.jpg', data=Row(Iptc4xmpExt:CountryCode=None, Iptc4xmpExt:CountryName=None, Iptc4xmpExt:ProvinceState=None, Iptc4xmpExt:WorldRegion=None, ac:accessURI='https://botanydb.colorado.edu/COLO_V/00806/00806653.jpg', ac:associatedSpecimenReference='https://botanydb.colorado.edu:443/collections/individual/index.php?occid=25', ac:attributionLogoURL=None, ac:bestQualityAccessURI=None, ac:bestQualityFormat=None, ac:caption=None, ac:captureDevice=None, ac:comments=None, ac:digitizationDate=None, ac:furtherInformationURL=None, ac:goodQualityAccessURI='https://botanydb.colorado.edu/COLO_V/00806/00806653.jpg', ac:hashFunction=None, ac:hashValue=None, ac:licenseLogoURL=None, ac:metadataCreator=None, ac:metadataLanguage='en', ac:metadataLanguageLiteral=None, ac:metadataProvider=None, ac:metadataProviderLiteral=None, ac:provider=None, ac:providerID=None, ac:providerLiteral=None, ac:providerManagedID='urn:uuid:e36c3d94-ea0d-40d2-96aa-0d3f

In [22]:
df.printSchema()

root
 |-- accessuri: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- Iptc4xmpExt:CountryCode: string (nullable = true)
 |    |-- Iptc4xmpExt:CountryName: string (nullable = true)
 |    |-- Iptc4xmpExt:ProvinceState: string (nullable = true)
 |    |-- Iptc4xmpExt:WorldRegion: string (nullable = true)
 |    |-- ac:accessURI: string (nullable = true)
 |    |-- ac:associatedSpecimenReference: string (nullable = true)
 |    |-- ac:attributionLogoURL: string (nullable = true)
 |    |-- ac:bestQualityAccessURI: string (nullable = true)
 |    |-- ac:bestQualityFormat: string (nullable = true)
 |    |-- ac:caption: string (nullable = true)
 |    |-- ac:captureDevice: string (nullable = true)
 |    |-- ac:comments: string (nullable = true)
 |    |-- ac:digitizationDate: string (nullable = true)
 |    |-- ac:furtherInformationURL: string (nullable = true)
 |    |-- ac:goodQualityAccessURI: string (nullable = true)
 |    |-- ac:hashFunction: string (nullable = true)
 |    |-

In [23]:
from pyspark.sql.types import *
def mk_field_set_from_df(df_schema):
    """Create a set of fieldname::type strings from a df schema"""
    field_set = set()
    prefix = ""
    for i in df_schema:
        #print(i)
        t = type(i.dataType)
        if t is not StringType and \
           t is not FloatType and \
           t is not IntegerType and \
           t is not TimestampType and \
           t is not BooleanType:
            #print(i.dataType)
            prefix = i.name
            
            # This only accommodates one level of nesting which is
            # consistant with our parquet-building technique. The
            # string manipulation on type makes it consistant with
            # the JSON returned by the API.
            for j in i.dataType:
                field_set.add(("{0}.{1}::{2}".format(prefix, j.name, 
                                            str(j.dataType)[:-4].lower())))

        else:
            field_set.add(("{0}::{1}".format(i.name, 
                                            str(i.dataType)[:-4].lower())))
    return field_set
df_field_set = mk_field_set_from_df(df.schema)
#print(df_field_set)

In [24]:
import requests

def mk_field_set_from_api():
    """Create a set of fieldname::type strings from the meta API endpoint"""
    meta_fields_records = (requests                                                                                                 
        .get("http://search.idigbio.org/v2/meta/fields/mediarecords")                                                 
        .json()                                                                                                  
    ) 
    field_set = set()                                                                                                              
    for k,v in meta_fields_records.items():
        if k == "data":                                                                                                             
            for kd,vd in v.items():                                                                                                 
                if vd.get("fieldName", False):                                                                                      
                    field_set.add("{0}::{1}".format(vd["fieldName"],
                                                        vd["type"]))
        elif v.get("fieldName", False):                                                                                               
            field_set.add("{0}::{1}".format(v["fieldName"], v["type"]))
        else:
            # non-data nested structures, only add top level key
            field_set.add("{0}::COMPLEX".format(k))
        
    return field_set
meta_field_set = mk_field_set_from_api()
#print(meta_field_set)

In [25]:
# The loaded df is a working data frame of course, what fields are in the
# meta endpoint that the df does not have? Are those fields currently 
# excluded in the parquet generation code or has Alex snuck in new
# structures lately?
#
# Note that ES calls things strings if they're lists of strings so even if
# something doesn't show up here, watch for scaler -> vector changes in the
# way data is represented in a field.
print("In meta endpoint, not df")
print("\n".join(sorted(meta_field_set - df_field_set)))
print("\nIn df, not meta endpoint")
print("\n".join(sorted(df_field_set - meta_field_set)))

In meta endpoint, not df
datemodified::date
indexData::COMPLEX
modified::date
recordids::string
records::string

In df, not meta endpoint
datemodified::timestamp
modified::timestamp
